In [ ]:
!pip install rdkit-pypi
!pip install git+https://github.com/samoturk/mol2vec

from google.colab import drive
drive.mount('/content/gdrive')

import os 
os.chdir('/content/gdrive/MyDrive/Project')

     |████████████████████████████████| 20.6 MB 11.2 MB/s 
  Cloning https://github.com/samoturk/mol2vec to /tmp/pip-req-build-3ou4dk48
  Running command git clone -q https://github.com/samoturk/mol2vec /tmp/pip-req-build-3ou4dk48
  Created wheel for mol2vec: filename=mol2vec-0.1-py3-none-any.whl size=14038 sha256=83ad890a7f5c7b25abfca84ea3ef181739fc79b501db58a95905a4d9df956404
  Stored in directory: /tmp/pip-ephem-wheel-cache-7omi36r3/wheels/80/d5/e1/90dc211e4a17780f99eaec43328470a70284cd731ef2a77034
Successfully built mol2vec
Mounted at /content/gdrive


In [ ]:

import pandas as pd
import numpy as np
from rdkit import Chem
from gensim.models import word2vec
#from mol2vec import features
from mol2vec import helpers
from mol2vec.features import mol2alt_sentence, mol2sentence,MolSentence,DfVec,sentences2vec


import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

import keras
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

In [ ]:
pKa=pd.read_csv('pKa.csv').iloc[:,:2]
hfus=pd.read_csv('hfus.csv').iloc[:,:2]

Y_pKa=pd.read_csv('pKa.csv').iloc[:,1]
Y_hfus=pd.read_csv('hfus.csv').iloc[:,1]

x_pKa=pd.read_csv('pKa.csv').iloc[:,0]
x_hfus=pd.read_csv('hfus.csv').iloc[:,0]

pKa.columns=['smiles','target']
hfus.columns=['smiles','target']
pKa=pKa.astype(object)
hfus=hfus.astype(object)

pKa.head()
hfus.head()

pKa['mol']=pKa['smiles'].apply(lambda x_pKa: Chem.MolFromSmiles(x_pKa))
hfus['mol']=hfus['smiles'].apply(lambda x_hfus: Chem.MolFromSmiles(x_hfus))

model=word2vec.Word2Vec.load('model_300dim.pkl')
pKa['sentence']= None
hfus['sentence']= None

for i in range(pKa.shape[0]):
  try:
    m1=pKa['mol'][i]
    pKa.loc[i,'sentence']=MolSentence(mol2alt_sentence(m1,1))
  except:
    #pKa.loc[i,'sentence']=MolSentence(mol2alt_sentence(m1,1))
    print('skipped:{}'.format(pKa['smiles'][i]))
pKa.dropna(inplace=True)
pKa.head()

pKa['mol2vec']=[DfVec(x) for x in sentences2vec(pKa['sentence'],model,unseen='UNK')]
X_pka=np.array([x.vec for x in pKa['mol2vec']])
y_pka=pKa['target'].values.astype(np.float32)


for i in range(hfus.shape[0]):
  try:
    m2=hfus['mol'][i]
    hfus.loc[i,'sentence']=MolSentence(mol2alt_sentence(m2,1))
  except:
    #hfus.loc[i,'sentence']=MolSentence(mol2alt_sentence(m2,1))
    print('skipped:{}'.format(hfus['smiles'][i]))
hfus.dropna(inplace=True)
hfus.head()

hfus['mol2vec']=[DfVec(x) for x in sentences2vec(hfus['sentence'],model,unseen='UNK')]
X_hfus=np.array([x.vec for x in hfus['mol2vec']])
y_hfus=hfus['target'].values.astype(np.float32)

X_pka_train, X_pka_tmp, y_pka_train, y_pka_tmp = train_test_split(X_pka,y_pka, test_size=0.2)
X_pka_valid, X_pka_test, y_pka_valid, y_pka_test = train_test_split(X_pka_tmp,y_pka_tmp, test_size=0.5)
X_hfus_train, X_hfus_tmp, y_hfus_train, y_hfus_tmp = train_test_split(X_hfus,y_hfus, test_size=0.2)
X_hfus_valid, X_hfus_test, y_hfus_valid, y_hfus_test = train_test_split(X_hfus_tmp,y_hfus_tmp, test_size=0.5)



skipped:O=c1nnnc2ccccc12
skipped:O=c1c2ccccc2cc3ccccc13


Linear Regression

In [ ]:
Linregr_pka = linear_model.ElasticNet(alpha=0.001,random_state=400,l1_ratio=0.1)
Linregr_pka.fit(X_pka_train, y_pka_train)
Linregr_hfus = linear_model.ElasticNet(alpha=0.001,random_state=400,l1_ratio=0.1)
Linregr_hfus.fit(X_hfus_train, y_hfus_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.201e+03, tolerance: 1.662e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.644e+03, tolerance: 8.358e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


ElasticNet(alpha=0.001, l1_ratio=0.1, random_state=400)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

#model1 = Ridge()
#cv1 = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
#space1 = dict()
#space1['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
#space1['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
#space1['fit_intercept'] = [True, False]
#space1['normalize'] = [True, False]
#search1 = GridSearchCV(model1, space1, scoring='neg_mean_absolute_error', n_jobs=-1)
#result1 = search1.fit(X_pka_valid, y_pka_valid)

#model2 = Ridge()
#cv2 = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
#space2 = dict()
#space2['solver'] = ['svd', 'cholesky', 'lsqr', 'sag']
#space2['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
#space2['fit_intercept'] = [True, False]
#space2['normalize'] = [True, False]
#search2 = GridSearchCV(model2, space2, scoring='neg_mean_absolute_error', n_jobs=-1)
#result2 = search2.fit(X_hfus_valid, y_hfus_valid)

print('Best Score for pka: %s' % result1.best_score_)
print('Best Hyperparameters for pka: %s' % result1.best_params_)

print('Best Score for hfus: %s' % result2.best_score_)
print('Best Hyperparameters for hfus: %s' % result2.best_params_)

Linregr_pka = linear_model.ElasticNet(alpha=1,random_state=400,l1_ratio=0.1)
Linregr_pka.fit(X_pka_train, y_pka_train)
Linregr_hfus = linear_model.ElasticNet(alpha=100,random_state=400,l1_ratio=0.1)
Linregr_hfus.fit(X_hfus_train, y_hfus_train)

y_Linpred_valid_pka = Linregr_pka.predict(X_pka_valid)
print('Coefficient of determination for validation case for pKa: {}'.format(r2_score(y_pka_valid,y_Linpred_valid_pka)))
y_Linpred_valid_hfus = Linregr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case for Heat of fusion: {}'.format(r2_score(y_hfus_valid,y_Linpred_valid_hfus)))

NameError: ignored

In [ ]:
y_Linpred_test_pka = Linregr_pka.predict(X_pka_test)
print('Coefficient of determination for test case for pKa: {}'.format(r2_score(y_pka_test,y_Linpred_test_pka)))
y_Linpred_test_hfus = Linregr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case for Heat of fusion: {}'.format(r2_score(y_hfus_test,y_Linpred_test_hfus)))

Coefficient of determination for test case for pKa: 0.404272166513869
Coefficient of determination for test case for Heat of fusion: 0.22652395190534969


Polynomial Regression

In [ ]:
pf_pka=PolynomialFeatures(degree=2)
pka_new=pf_pka.fit_transform(X_pka_train)
pf_hfus=PolynomialFeatures(degree=2)
hfus_new=pf_hfus.fit_transform(X_hfus_train)

In [ ]:
polyregr_pka = linear_model.ElasticNet(alpha=0.088,random_state=400,l1_ratio=0.1,max_iter=1000)
polyregr_pka.fit(pka_new, y_pka_train)
polyregr_hfus = linear_model.ElasticNet(alpha=0.3,random_state=400,l1_ratio=0.1,max_iter=1000)
polyregr_hfus.fit(hfus_new, y_hfus_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.018e+02, tolerance: 1.662e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.731e+03, tolerance: 8.358e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


ElasticNet(alpha=0.3, l1_ratio=0.1, random_state=400)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

#Grid Search Hyperparameter tuning commented due to computational restriction(execution crashes due to shortage of RAM)

#param_grid1 = [{'poly__degree': [2, 3, 4, 5, 6, 7, 8, 9]}]
#pipeline1 = Pipeline(steps=[('poly', PolynomialFeatures()), ('ridge', Ridge())])
#grid_search1 = GridSearchCV(pipeline1, param_grid1, cv=5,scoring='neg_mean_squared_error', return_train_score=True)
#grid_search1.fit(X_pka_valid, y_pka_valid)

#param_grid2 = [{'poly__degree': [2, 3, 4, 5, 6, 7, 8, 9]}]
#pipeline2 = Pipeline(steps=[('poly', PolynomialFeatures()), ('ridge', Ridge())])
#grid_search2 = GridSearchCV(pipeline2, param_grid2, cv=5,scoring='neg_mean_squared_error', return_train_score=True)
#grid_search2.fit(X_hfus_valid, y_hfus_valid)

#grid_search1.best_params_
#grid_search2.best_params_

y_polypred_valid_pka = polyregr_pka.predict(pf_pka.transform(X_pka_valid))
y_polypred_valid_hfus = polyregr_hfus.predict(pf_hfus.transform(X_hfus_valid))
print('Coefficient of determination for validation case for pKa: {}'.format(r2_score(y_pka_valid,y_polypred_valid_pka)))
print('Coefficient of determination for validation case for Heat of fusion: {}'.format(r2_score(y_hfus_valid,y_polypred_valid_hfus)))

Coefficient of determination for validation case for pKa: -0.006235051736527941
Coefficient of determination for validation case for Heat of fusion: 0.6987396938593199


In [ ]:
y_polypred_test_pka = polyregr_pka.predict(pf_pka.transform(X_pka_test))
y_polypred_test_hfus = polyregr_hfus.predict(pf_hfus.transform(X_hfus_test))
print('Coefficient of determination for test case for pKa: {}'.format(r2_score(y_pka_test,y_polypred_test_pka)))
print('Coefficient of determination for test case for Heat of fusion: {}'.format(r2_score(y_hfus_test,y_polypred_test_hfus)))

Coefficient of determination for test case for pKa: -0.32227223540687544
Coefficient of determination for test case for Heat of fusion: 0.41893394817968355


Support Vector Regression with Linear Kernel

In [ ]:
svr_pka_linear = SVR(kernel='linear',C=1.842,epsilon=0.002)
svr_pka_linear.fit(X_pka_train, y_pka_train)

svr_hfus_linear = SVR(kernel='linear',C=1.389,epsilon=0.002)
svr_hfus_linear.fit(X_hfus_train, y_hfus_train)


SVR(C=1.389, epsilon=0.002, kernel='linear')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

#Tuning Hyperparameters using Grid Search
#param_grid1 = {'C': [0.1, 1, 10, 100, 1000],'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['linear']              }
 
#grid1 = GridSearchCV(SVC(), param_grid1, refit = True, verbose = 3)
#grid1.fit(X_pka_valid.astype('int'), y_pka_valid.astype('int'))

#param_grid2= {'C': [0.1, 1, 10, 100, 1000],'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['linear']              }
 
#grid2 = GridSearchCV(SVC(), param_grid2, refit = True, verbose = 3)
#grid2.fit(X_hfus_valid.astype('int'), y_hfus_valid.astype('int'))

#print(grid1.best_params_)
#print(grid1.best_estimator_)
#print(grid2.best_params_)
#print(grid2.best_estimator_)

svr_pka_linear = SVR(kernel='linear',C=1000,gamma=0.0001)
svr_pka_linear.fit(X_pka_train, y_pka_train)

svr_hfus_linear = SVR(kernel='linear',C=10,gamma=0.0001)
svr_hfus_linear.fit(X_hfus_train, y_hfus_train)

y_pred_linear_pka_valid = svr_pka_linear.predict(X_pka_valid)
y_pred_linear_hfus_valid = svr_hfus_linear.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_linear_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_linear_hfus_valid)))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.303 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.333 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.242 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.375 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.438 total time=   0.0s
[CV 1/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.303 total time=   0.0s
[CV 2/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.333 total time=   0.0s
[CV 3/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.242 total time=   0.0s
[CV 4/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.375 total time=   0.0s
[CV 5/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.438 total time=   0.0s
[CV 1/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.303 total time=   0.0s
[CV 2/5] END ..C=0.1, gamma=0.01, kernel=linear

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


[CV 1/5] END C=0.1, gamma=0.0001, kernel=linear;, score=0.303 total time=   0.0s
[CV 2/5] END C=0.1, gamma=0.0001, kernel=linear;, score=0.333 total time=   0.0s
[CV 3/5] END C=0.1, gamma=0.0001, kernel=linear;, score=0.242 total time=   0.0s
[CV 4/5] END C=0.1, gamma=0.0001, kernel=linear;, score=0.375 total time=   0.0s
[CV 5/5] END C=0.1, gamma=0.0001, kernel=linear;, score=0.438 total time=   0.0s
[CV 1/5] END .......C=1, gamma=1, kernel=linear;, score=0.303 total time=   0.0s
[CV 2/5] END .......C=1, gamma=1, kernel=linear;, score=0.303 total time=   0.0s
[CV 3/5] END .......C=1, gamma=1, kernel=linear;, score=0.242 total time=   0.0s
[CV 4/5] END .......C=1, gamma=1, kernel=linear;, score=0.344 total time=   0.0s
[CV 5/5] END .......C=1, gamma=1, kernel=linear;, score=0.438 total time=   0.0s
[CV 1/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.303 total time=   0.0s
[CV 2/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.303 total time=   0.0s
[CV 3/5] END .....C=1, gamma

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


[CV 1/5] END ..C=1, gamma=0.0001, kernel=linear;, score=0.000 total time=   0.0s
[CV 2/5] END ..C=1, gamma=0.0001, kernel=linear;, score=0.067 total time=   0.0s
[CV 3/5] END ..C=1, gamma=0.0001, kernel=linear;, score=0.267 total time=   0.0s
[CV 4/5] END ..C=1, gamma=0.0001, kernel=linear;, score=0.067 total time=   0.0s
[CV 5/5] END ..C=1, gamma=0.0001, kernel=linear;, score=0.067 total time=   0.0s
[CV 1/5] END ......C=10, gamma=1, kernel=linear;, score=0.000 total time=   0.0s
[CV 2/5] END ......C=10, gamma=1, kernel=linear;, score=0.067 total time=   0.0s
[CV 3/5] END ......C=10, gamma=1, kernel=linear;, score=0.267 total time=   0.0s
[CV 4/5] END ......C=10, gamma=1, kernel=linear;, score=0.067 total time=   0.0s
[CV 5/5] END ......C=10, gamma=1, kernel=linear;, score=0.067 total time=   0.0s
[CV 1/5] END ....C=10, gamma=0.1, kernel=linear;, score=0.000 total time=   0.0s
[CV 2/5] END ....C=10, gamma=0.1, kernel=linear;, score=0.067 total time=   0.0s
[CV 3/5] END ....C=10, gamma

In [ ]:
y_pred_linear_pka_test = svr_pka_linear.predict(X_pka_test)
y_pred_linear_hfus_test = svr_hfus_linear.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_linear_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test, y_pred_linear_hfus_test)))

Coefficient of determination for test case: -0.05383533319727918
Coefficient of determination for test case: 0.4722872178664237


Support Vector Regression with Polynomial Kernel


In [ ]:
svr_pka_poly = SVR(kernel='poly',degree=2,C=10,epsilon=0.01)
svr_pka_poly.fit(X_pka_train, y_pka_train)

svr_hfus_poly = SVR(kernel='poly',degree=3,C=100,epsilon=0.05)
svr_hfus_poly.fit(X_hfus_train, y_hfus_train)


SVR(C=100, epsilon=0.05, kernel='poly')

In [ ]:
#hyperparameter tuning commented to preserve output of validation set
#from sklearn.model_selection import GridSearchCV
#from sklearn.svm import SVC
#param_grid1 = {'C': [0.1, 1, 10, 100, 1000],
              #'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['poly']
              #}
 
#grid1 = GridSearchCV(SVC(), param_grid1, refit = True, verbose = 3)
#grid1.fit(X_pka_valid.astype('int'), y_pka_valid.astype('int'))

#param_grid2= {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['poly']
              }
 
#grid2 = GridSearchCV(SVC(), param_grid2, refit = True, verbose = 3)
#grid2.fit(X_hfus_valid.astype('int'), y_hfus_valid.astype('int'))

#print(grid1.best_estimator_)
#print(grid2.best_estimator_)

#svr_pka_poly = SVR(kernel='poly',degree=2,C=10,gamma=0.001)
#svr_pka_poly.fit(X_pka_train, y_pka_train)

#svr_hfus_poly = SVR(kernel='poly',degree=3,C=0.1,gamma=0.001)
#svr_hfus_poly.fit(X_hfus_train, y_hfus_train)

y_pred_poly_pka_valid = svr_pka_poly.predict(X_pka_valid)
y_pred_poly_hfus_valid = svr_hfus_poly.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_poly_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_poly_hfus_valid)))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.212 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.242 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.212 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.125 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.156 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.1, kernel=poly;, score=0.212 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.1, kernel=poly;, score=0.242 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=poly;, score=0.212 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=0.1, kernel=poly;, score=0.125 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=0.1, kernel=poly;, score=0.156 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=poly;, score=0.212 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=poly

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,



[CV 4/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.125 total time=   0.0s
[CV 5/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.156 total time=   0.0s
[CV 1/5] END .........C=1, gamma=1, kernel=poly;, score=0.212 total time=   0.0s
[CV 2/5] END .........C=1, gamma=1, kernel=poly;, score=0.242 total time=   0.0s
[CV 3/5] END .........C=1, gamma=1, kernel=poly;, score=0.212 total time=   0.0s
[CV 4/5] END .........C=1, gamma=1, kernel=poly;, score=0.125 total time=   0.0s
[CV 5/5] END .........C=1, gamma=1, kernel=poly;, score=0.156 total time=   0.0s
[CV 1/5] END .......C=1, gamma=0.1, kernel=poly;, score=0.212 total time=   0.0s
[CV 2/5] END .......C=1, gamma=0.1, kernel=poly;, score=0.242 total time=   0.0s
[CV 3/5] END .......C=1, gamma=0.1, kernel=poly;, score=0.212 total time=   0.0s
[CV 4/5] END .......C=1, gamma=0.1, kernel=poly;, score=0.125 total time=   0.0s
[CV 5/5] END .......C=1, gamma=0.1, kernel=poly;, score=0.156 total time=   0.0s
[CV 1/5] END ......C=1, gam

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


[CV 4/5] END ....C=0.1, gamma=0.01, kernel=poly;, score=0.067 total time=   0.0s
[CV 5/5] END ....C=0.1, gamma=0.01, kernel=poly;, score=0.067 total time=   0.0s
[CV 1/5] END ...C=0.1, gamma=0.001, kernel=poly;, score=0.067 total time=   0.0s
[CV 2/5] END ...C=0.1, gamma=0.001, kernel=poly;, score=0.133 total time=   0.0s
[CV 3/5] END ...C=0.1, gamma=0.001, kernel=poly;, score=0.067 total time=   0.0s
[CV 4/5] END ...C=0.1, gamma=0.001, kernel=poly;, score=0.067 total time=   0.0s
[CV 5/5] END ...C=0.1, gamma=0.001, kernel=poly;, score=0.133 total time=   0.0s
[CV 1/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.067 total time=   0.0s
[CV 2/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.067 total time=   0.0s
[CV 3/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.067 total time=   0.0s
[CV 4/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.067 total time=   0.0s
[CV 5/5] END ..C=0.1, gamma=0.0001, kernel=poly;, score=0.067 total time=   0.0s
[CV 1/5] END .........C=1, g

In [ ]:
y_pred_poly_pka_test = svr_pka_poly.predict(X_pka_test)
y_pred_poly_hfus_test = svr_hfus_poly.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_poly_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test, y_pred_poly_hfus_test)))

Coefficient of determination for test case: -0.034754160633276854
Coefficient of determination for test case: 0.7096128170010454


Support Vector regression with RBF Kernel

In [ ]:
svr_pka = SVR(kernel='rbf',C=60,epsilon=0.01)
svr_pka.fit(X_pka_train, y_pka_train)

svr_hfus = SVR(kernel='rbf',C=60,epsilon=0.01)
svr_hfus.fit(X_hfus_train, y_hfus_train)

SVR(C=60, epsilon=0.01)

In [ ]:
#hyperparameter tuning commented to preserve output of validation set
#from sklearn.model_selection import GridSearchCV
#from sklearn.svm import SVC
#param_grid1 = {'C': [0.1, 1, 10, 100, 1000],
            #  'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['rbf']
            #  }
 
#grid1 = GridSearchCV(SVC(), param_grid1, refit = True, verbose = 3)
g#rid1.fit(X_pka_valid.astype('int'), y_pka_valid.astype('int'))

#param_grid2= {'C': [0.1, 1, 10, 100, 1000],
              #'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['rbf']
              #}
 
#grid2 = GridSearchCV(SVC(), param_grid2, refit = True, verbose = 3)
#grid2.fit(X_hfus_valid.astype('int'), y_hfus_valid.astype('int'))

#print(grid1.best_estimator_)
#print(grid2.best_estimator_)

#svr_pka = SVR(kernel='rbf',C=100,gamma=0.0001)
#svr_pka.fit(X_pka_train, y_pka_train)

#svr_hfus = SVR(kernel='rbf',C=10,epsilon=0.01)
#svr_hfus.fit(X_hfus_train, y_hfus_train)

y_pred_rbf_pka_valid = svr_pka.predict(X_pka_valid)
y_pred_rbf_hfus_valid = svr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_rbf_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_rbf_hfus_valid)))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.152 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.152 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.152 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.125 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.156 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.152 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.152 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.152 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.125 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.156 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.152 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


[CV 4/5] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.125 total time=   0.0s
[CV 5/5] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.156 total time=   0.0s
[CV 1/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.152 total time=   0.0s
[CV 2/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.152 total time=   0.0s
[CV 3/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.152 total time=   0.0s
[CV 4/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.125 total time=   0.0s
[CV 5/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.156 total time=   0.0s
[CV 1/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.152 total time=   0.0s
[CV 2/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.152 total time=   0.0s
[CV 3/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.152 total time=   0.0s
[CV 4/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.125 total time=   0.0s
[CV 5/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.156 total time=   0.0s
[CV 1/5] END ........C=1, ga

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


[CV 1/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.067 total time=   0.0s
[CV 2/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.067 total time=   0.0s
[CV 3/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.067 total time=   0.0s
[CV 4/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.067 total time=   0.0s
[CV 5/5] END ...C=0.1, gamma=0.0001, kernel=rbf;, score=0.000 total time=   0.0s
[CV 1/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.067 total time=   0.0s
[CV 2/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.067 total time=   0.0s
[CV 3/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.067 total time=   0.0s
[CV 4/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.067 total time=   0.0s
[CV 5/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.000 total time=   0.0s
[CV 1/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.067 total time=   0.0s
[CV 2/5] END ........C=1, gamma=0.1, kernel=rbf;, score=0.067 total time=   0.0s
[CV 3/5] END ........C=1, ga

In [ ]:
y_pred_rbf_pka_test = svr_pka.predict(X_pka_test)
y_pred_rbf_hfus_test = svr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_rbf_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test, y_pred_rbf_hfus_test)))

Coefficient of determination for test case: 0.4602785087598862
Coefficient of determination for test case: 0.8584542021061013


Decision Trees

In [ ]:
dt_regr_pka = DecisionTreeRegressor(max_depth=13,criterion='squared_error',random_state=200)
dt_regr_hfus = DecisionTreeRegressor(max_depth=9,criterion='squared_error',random_state=200)
dt_regr_pka.fit(X_pka_train, y_pka_train)
dt_regr_hfus.fit(X_hfus_train, y_hfus_train)

DecisionTreeRegressor(max_depth=9, random_state=200)

In [ ]:
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
  
param_dist1 = {"max_depth": [3,4,5,6,7, None],"max_features": range(1, 9),"min_samples_leaf": range(1, 9),"criterion": ["squared_error"]}
tree1 = DecisionTreeClassifier()
tree_cv1 = RandomizedSearchCV(tree, param_dist1, cv = 5)  
tree_cv1.fit(X_pka_valid.astype('int'), y_pka_valid.astype('int'))

param_dist2 = {"max_depth": [3,4,5,6,7, None],"max_features": range(1, 9),"min_samples_leaf": range(1, 9),"criterion": ["squared_error"]}
tree2 = DecisionTreeClassifier()
tree_cv2 = RandomizedSearchCV(tree, param_dist, cv = 5)  
tree_cv2.fit(X_hfus_valid.astype('int'), y_hfus_valid.astype('int'))
 

print("Tuned Decision Tree Parameters for pka: {}".format(tree_cv1.best_params_))
print("Best score is {}".format(tree_cv1.best_score_))

print("Tuned Decision Tree Parameters for Hfu: {}".format(tree_cv2.best_params_))
print("Best score is {}".format(tree_cv2.best_score_))

dt_regr_pka = DecisionTreeRegressor(max_depth=4,criterion="squared_error")
dt_regr_hfus = DecisionTreeRegressor(max_depth=3,criterion="squared_error")
dt_regr_pka.fit(X_pka_train, y_pka_train)
dt_regr_hfus.fit(X_hfus_train, y_hfus_train)


y_pred_dtse_pka_valid = dt_regr_pka.predict(X_pka_valid)
y_pred_dtse_hfus_valid = dt_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_dtse_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_dtse_hfus_valid)))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "

Tuned Decision Tree Parameters for pka: {'min_samples_leaf': 1, 'max_features': 2, 'max_depth': 4, 'criterion': 'gini'}
Best score is 0.19564393939393937
Tuned Decision Tree Parameters for Hfu: {'criterion': 'gini', 'max_depth': None, 'max_features': 6, 'min_samples_leaf': 6}
Best score is 0.12
Coefficient of determination for validation case: 0.15546002268580983
Coefficient of determination for validation case: 0.6719873756869306


In [ ]:
y_pred_dtse_pka_test = dt_regr_pka.predict(X_pka_test)
y_pred_dtse_hfus_test = dt_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_dtse_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test, y_pred_dtse_hfus_test)))

Coefficient of determination for test case: 0.13506516714036831
Coefficient of determination for test case: 0.6840027472696013


Ridge Regression

In [ ]:
ridge_regr_pka = Ridge(alpha=8)
ridge_regr_hfus = Ridge(alpha=8)
ridge_regr_pka.fit(X_pka_train, y_pka_train)
ridge_regr_hfus.fit(X_hfus_train, y_hfus_train)

Ridge(alpha=8)

In [ ]:
ridge_params = {'alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}
Ridge1= GridSearchCV(linear_model.Ridge(),param_grid=ridge_params).fit(X_pka_valid, y_pka_valid)
Ridge2= GridSearchCV(linear_model.Ridge(),param_grid=ridge_params).fit(X_hfus_valid, y_hfus_valid)
print(Ridge1.best_estimator_)
print(Ridge1.best_estimator_)

ridge_regr_pka = Ridge(alpha=500)
ridge_regr_hfus = Ridge(alpha=500)
ridge_regr_pka.fit(X_pka_train, y_pka_train)
ridge_regr_hfus.fit(X_hfus_train, y_hfus_train)


Ridge(alpha=500)
Ridge(alpha=500)


Ridge(alpha=500)

In [ ]:
y_pred_ridreg_pka_valid = ridge_regr_pka.predict(X_pka_valid)
y_pred_ridreg_hfus_valid = ridge_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_ridreg_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_ridreg_hfus_valid)))

Coefficient of determination for validation case: 0.4660912743628227
Coefficient of determination for validation case: 0.7379529138383092


In [ ]:
y_pred_ridreg_pka_test = ridge_regr_pka.predict(X_pka_test)
y_pred_ridreg_hfus_test = ridge_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_ridreg_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_ridreg_hfus_test)))

Coefficient of determination for test case: 0.335247069777004
Coefficient of determination for test case: 0.8348982206031852


Kernel Ridge - Linear

In [ ]:
kernelridge_linear_regr_pka = KernelRidge(alpha=1.52,kernel='linear')
kernelridge_linear_regr_hfus = KernelRidge(alpha=3.23,kernel='linear')
kernelridge_linear_regr_pka.fit(X_pka_train, y_pka_train)
kernelridge_linear_regr_hfus.fit(X_hfus_train, y_hfus_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:193: LinAlgWarning: Ill-conditioned matrix (rcond=9.06422e-09): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)


KernelRidge(alpha=3.23)

In [ ]:
#hyperparameter tuning commented to preserve output of validation set
#max1=0.0
#max2=0.0
#alpha1=0.0
#alpha2=0.0
#for i in range(1000):
  #j=float(i)/100.0
  #kernelridge_linear_regr_pka = KernelRidge(alpha=j,kernel='linear')
  #kernelridge_linear_regr_hfus = KernelRidge(alpha=j,kernel='linear')
  #kernelridge_linear_regr_pka.fit(X_pka_train, y_pka_train)
  #kernelridge_linear_regr_hfus.fit(X_hfus_train, y_hfus_train)
  #y_pred_kerreg_linear_pka_valid = kernelridge_linear_regr_pka.predict(X_pka_valid)
 # y_pred_kerreg_linear_hfus_valid = kernelridge_linear_regr_hfus.predict(X_hfus_valid)
  #r2_score1=r2_score(y_pka_valid, y_pred_kerreg_linear_pka_valid)
  #r2_score2=r2_score(y_hfus_valid, y_pred_kerreg_linear_hfus_valid)
  #if(max1<r2_score1):
   # alpha1=j
  #if(max2<r2_score2):
    #alpha2=j

#kernelridge_linear_regr_pka = KernelRidge(alpha=alpha1,kernel='linear')
#kernelridge_linear_regr_hfus = KernelRidge(alpha=alpha2,kernel='linear')
#kernelridge_linear_regr_pka.fit(X_pka_train, y_pka_train)
#kernelridge_linear_regr_hfus.fit(X_hfus_train, y_hfus_train)


y_pred_kerreg_linear_pka_valid = kernelridge_linear_regr_pka.predict(X_pka_valid)
y_pred_kerreg_linear_hfus_valid = kernelridge_linear_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_kerreg_linear_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_kerreg_linear_hfus_valid)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual prob

Coefficient of determination for validation case: 0.21635473995412435
Coefficient of determination for validation case: 0.6765968485579485


In [ ]:
y_pred_kerreg_linear_pka_test = kernelridge_linear_regr_pka.predict(X_pka_test)
y_pred_kerreg_linear_hfus_test = kernelridge_linear_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_kerreg_linear_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_kerreg_linear_hfus_test)))

Coefficient of determination for test case: -0.141028193553993
Coefficient of determination for test case: 0.6829597835430199


Kernel Ridge-rbf

In [ ]:
kernelridge_rbf_regr_pka = KernelRidge(alpha=0.01,kernel='rbf')
kernelridge_rbf_regr_hfus = KernelRidge(alpha=0.01,kernel='rbf')
kernelridge_rbf_regr_pka.fit(X_pka_train, y_pka_train)
kernelridge_rbf_regr_hfus.fit(X_hfus_train, y_hfus_train)

KernelRidge(alpha=0.01, kernel='rbf')

In [ ]:
max1=0.0
max2=0.0
alpha1=0.0
alpha2=0.0
for i in range(1000):
  j=float(i)/100.0
  kernelridge_linear_regr_pka = KernelRidge(alpha=j,kernel='rbf')
  kernelridge_linear_regr_hfus = KernelRidge(alpha=j,kernel='rbf')
  kernelridge_linear_regr_pka.fit(X_pka_train, y_pka_train)
  kernelridge_linear_regr_hfus.fit(X_hfus_train, y_hfus_train)
  y_pred_kerreg_linear_pka_valid = kernelridge_linear_regr_pka.predict(X_pka_valid)
  y_pred_kerreg_linear_hfus_valid = kernelridge_linear_regr_hfus.predict(X_hfus_valid)
  r2_score1=r2_score(y_pka_valid, y_pred_kerreg_linear_pka_valid)
  r2_score2=r2_score(y_hfus_valid, y_pred_kerreg_linear_hfus_valid)
  if(max1<r2_score1):
    alpha1=j
  if(max2<r2_score2):
    alpha2=j

y_pred_kerreg_rbf_pka_valid = kernelridge_rbf_regr_pka.predict(X_pka_valid)
y_pred_kerreg_rbf_hfus_valid = kernelridge_rbf_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_kerreg_rbf_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_kerreg_rbf_hfus_valid)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "


Coefficient of determination for validation case: 0.1725200774430392
Coefficient of determination for validation case: 0.727884485627601


In [ ]:
y_pred_kerreg_rbf_pka_test = kernelridge_rbf_regr_pka.predict(X_pka_test)
y_pred_kerreg_rbf_hfus_test = kernelridge_rbf_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_kerreg_rbf_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_kerreg_rbf_hfus_test)))

Coefficient of determination for test case: -0.2669764674357493
Coefficient of determination for test case: 0.43541263797205887


Kernel Ridge-polynomial

In [ ]:
kernelridge_poly_regr_pka = KernelRidge(alpha=0.001,kernel='poly',degree=3)
kernelridge_poly_regr_hfus = KernelRidge(alpha=0.001,kernel='poly',degree=3)
kernelridge_poly_regr_pka.fit(X_pka_train, y_pka_train)
kernelridge_poly_regr_hfus.fit(X_hfus_train, y_hfus_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "


KernelRidge(alpha=0.001, kernel='poly')

In [ ]:
#hyperparameter tuning commented to preserve output of validation set
#max1=0.0
#max2=0.0
#alpha1=0.0
#alpha2=0.0
#for i in range(1000):
  #j=float(i)/1000.0
  #kernelridge_linear_regr_pka = KernelRidge(alpha=j,kernel='poly',degree=3)
  #kernelridge_linear_regr_hfus = KernelRidge(alpha=j,kernel='poly',degree=3)
  #kernelridge_linear_regr_pka.fit(X_pka_train, y_pka_train)
  #kernelridge_linear_regr_hfus.fit(X_hfus_train, y_hfus_train)
  #y_pred_kerreg_linear_pka_valid = kernelridge_linear_regr_pka.predict(X_pka_valid)
  #y_pred_kerreg_linear_hfus_valid = kernelridge_linear_regr_hfus.predict(X_hfus_valid)
  #r2_score1=r2_score(y_pka_valid, y_pred_kerreg_linear_pka_valid)
  #r2_score2=r2_score(y_hfus_valid, y_pred_kerreg_linear_hfus_valid)
  #if(max1<r2_score1):
    #alpha1=j
  #if(max2<r2_score2):
   # alpha2=j
y_pred_kerreg_poly_pka_valid = kernelridge_poly_regr_pka.predict(X_pka_valid)
y_pred_kerreg_poly_hfus_valid = kernelridge_poly_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_kerreg_poly_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_kerreg_poly_hfus_valid)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual prob

Coefficient of determination for validation case: -19.622759860683523
Coefficient of determination for validation case: 0.5123886558700104


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:193: LinAlgWarning: Ill-conditioned matrix (rcond=1.4273e-09): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "


In [ ]:
y_pred_kerreg_poly_pka_test = kernelridge_poly_regr_pka.predict(X_pka_test)
y_pred_kerreg_poly_hfus_test = kernelridge_poly_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_kerreg_poly_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_kerreg_poly_hfus_test)))

Coefficient of determination for test case: -4.915053032589205
Coefficient of determination for test case: 0.15509544009325904


Random forests

In [ ]:
rf_regr_pka = RandomForestRegressor(n_estimators=100,max_depth=18, random_state=200, criterion='squared_error',ccp_alpha=0.0005)
rf_regr_hfus = RandomForestRegressor(n_estimators=100,max_depth=17, random_state=200, criterion='squared_error',ccp_alpha=0.0005)
rf_regr_pka.fit(X_pka_train, y_pka_train)
rf_regr_hfus.fit(X_hfus_train, y_hfus_train)  

RandomForestRegressor(ccp_alpha=0.0005, max_depth=17, random_state=200)

In [ ]:
#from sklearn.ensemble import RandomForestRegressor
#rf1 = RandomForestRegressor(random_state = 42)
#from pprint import pprint
#print('Parameters currently in use:\n')
#pprint(rf1.get_params())
#from sklearn.model_selection import RandomizedSearchCV
#n_estimators1 = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
#max_features1 = ['auto', 'sqrt']
#max_depth1 = [int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth1.append(None)
#min_samples_split1 = [2, 5, 10]
#min_samples_leaf1 = [1, 2, 4]
#bootstrap1 = [True, False]

#random_grid1 = {'n_estimators': n_estimators1,max_features': max_features1,'max_depth': max_depth1,,min_samples_split': min_samples_split1,'min_samples_leaf': min_samples_leaf1,  'bootstrap': bootstrap1}
#rf1= RandomForestRegressor()
#rf_random1 = RandomizedSearchCV(estimator = rf1, param_distributions = random_grid1, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
#rf_random1.fit(X_pka_valid, y_pka_valid)
#print(rf_random1.best_params_)

#rf2 = RandomForestRegressor(random_state = 42)
#from pprint import pprint
#print('Parameters currently in use:\n')
#pprint(rf2.get_params())
#from sklearn.model_selection import RandomizedSearchCV
#n_estimators2 = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
#max_features2 = ['auto', 'sqrt']
#max_depth2 = [int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth2.append(None)
#min_samples_split2 = [2, 5, 10]
#min_samples_leaf2 = [1, 2, 4]
#bootstrap2 = [True, False]

#random_grid2 = {'n_estimators': n_estimators1,'max_features': max_features1,'max_depth': max_depth1,'min_samples_split': min_samples_split1,'min_samples_leaf': min_samples_leaf1,'bootstrap': bootstrap1}
#rf2= RandomForestRegressor()
#rf_random2 = RandomizedSearchCV(estimator = rf2, param_distributions = random_grid2, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
#rf_random2.fit(X_pka_valid, y_pka_valid)
#print(rf_random2.best_params_)


y_pred_rf_pka_valid = rf_regr_pka.predict(X_pka_valid)
y_pred_rf_hfus_valid = rf_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_rf_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_rf_hfus_valid)))

NameError: ignored

In [ ]:
y_pred_rf_pka_test = rf_regr_pka.predict(X_pka_test)
y_pred_rf_hfus_test = rf_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_rf_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_rf_hfus_test)))

Coefficient of determination for test case: 0.395713317476596
Coefficient of determination for test case: 0.5596254102238931


kNN

In [ ]:
knn_regr_pka = KNeighborsRegressor(n_neighbors=5,weights='uniform',p=2,metric='minkowski')
knn_regr_hfus = KNeighborsRegressor(n_neighbors=7,weights='uniform',p=2,metric='minkowski')
knn_regr_pka.fit(X_pka_train, y_pka_train)
knn_regr_hfus.fit(X_hfus_train, y_hfus_train)

KNeighborsRegressor(n_neighbors=7)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

leaf_size = list(range(1,10))
n_neighbors = list(range(1,10))
p=[1,2]
hyperparameters1 = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
knn_21 = KNeighborsClassifier()
clf1 = GridSearchCV(knn_21, hyperparameters1, cv=5)
best_model1 = clf1.fit(X_pka_valid.astype(int),y_pka_valid.astype(int))
print('Best leaf_size:', best_model1.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model1.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model1.best_estimator_.get_params()['n_neighbors'])

hyperparameters2 = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
knn_22 = KNeighborsClassifier()
clf2 = GridSearchCV(knn_22, hyperparameters2, cv=5)
best_model2 = clf2.fit(X_hfus_valid.astype(int),y_hfus_valid.astype(int))
print('Best leaf_size:', best_model2.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model2.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model2.best_estimator_.get_params()['n_neighbors'])

knn_regr_pka = KNeighborsRegressor(n_neighbors=5,weights='uniform',p=2,metric='minkowski')
knn_regr_hfus = KNeighborsRegressor(n_neighbors=7,weights='uniform',p=2,metric='minkowski')
knn_regr_pka.fit(X_pka_train, y_pka_train)
knn_regr_hfus.fit(X_hfus_train, y_hfus_train)

knn_regr_pka = KNeighborsRegressor(n_neighbors=9,weights='uniform',p=2,metric='minkowski')
knn_regr_hfus = KNeighborsRegressor(n_neighbors=7,weights='uniform',p=2,metric='minkowski')
knn_regr_pka.fit(X_pka_train, y_pka_train)
knn_regr_hfus.fit(X_hfus_train, y_hfus_train)


y_pred_knn_pka_valid = knn_regr_pka.predict(X_pka_valid)
y_pred_knn_poly_hfus_valid = knn_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_knn_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_knn_poly_hfus_valid)))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


Best leaf_size: 1
Best p: 1
Best n_neighbors: 1


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


Best leaf_size: 1
Best p: 2
Best n_neighbors: 2
Coefficient of determination for validation case: 0.32675635726356533
Coefficient of determination for validation case: 0.6359147968785214


In [ ]:
y_pred_knn_pka_test = knn_regr_pka.predict(X_pka_test)
y_pred_knn_hfus_test = knn_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_knn_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_knn_hfus_test)))

Coefficient of determination for test case: 0.36002578811254504
Coefficient of determination for test case: 0.8221640787309415


Gaussian Process Regression

In [ ]:
kernel = DotProduct() + WhiteKernel()
gpr_regr_pka = GaussianProcessRegressor(kernel=kernel,random_state=200,n_restarts_optimizer=10,alpha=0.025)
gpr_regr_hfus = GaussianProcessRegressor(kernel=kernel,random_state=200,n_restarts_optimizer=10,alpha=0.017)
gpr_regr_pka.fit(X_pka_train, y_pka_train)
gpr_regr_hfus.fit(X_hfus_train, y_hfus_train)

GaussianProcessRegressor(alpha=0.017,
                         kernel=DotProduct(sigma_0=1) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=10, random_state=200)

In [ ]:
y_pred_gpr_pka_valid = gpr_regr_pka.predict(X_pka_valid)
y_pred_gpr_poly_hfus_valid = gpr_regr_hfus.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_gpr_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_gpr_poly_hfus_valid)))

Coefficient of determination for validation case: 0.38044994717041036
Coefficient of determination for validation case: 0.5382277674222613


In [ ]:
y_pred_gpr_pka_test = gpr_regr_pka.predict(X_pka_test)
y_pred_gpr_hfus_test = gpr_regr_hfus.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_gpr_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_gpr_hfus_test)))

Coefficient of determination for test case: 0.39570483558307357
Coefficient of determination for test case: 0.7367026752359578


Neural Network

In [ ]:
pka_model = Sequential()
pka_model.add(Dense(200, input_dim=300, activation='relu'))
pka_model.add(Dense(200, input_dim=200, activation='relu'))
pka_model.add(Dense(200, input_dim=200, activation='relu'))
pka_model.add(Dense(1, activation='linear'))

hfus_model = Sequential()
hfus_model.add(Dense(200, input_dim=300, activation='relu'))
hfus_model.add(Dense(200, input_dim=200, activation='relu'))
hfus_model.add(Dense(200, input_dim=200, activation='relu'))
hfus_model.add(Dense(1, activation='linear'))

In [ ]:
tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.99, beta_2=0.999, amsgrad=False)
pka_model.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])

tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.99, beta_2=0.999, amsgrad=False)
hfus_model.compile(loss='mean_squared_error', optimizer='RMSprop', metrics=['mean_absolute_percentage_error'])
pka_model.fit(X_pka_train, y_pka_train, epochs=50, batch_size=32,validation_split=0.15,validation_data=None,verbose=1)
hfus_model.fit(X_hfus_train, y_hfus_train, epochs=50, batch_size=32,validation_split=0.15,validation_data=None,verbose=1)

Epoch 1/50
35/35 [==============================] - 2s 16ms/step - loss: 0.4623 - mean_absolute_percentage_error: 422007.2812 - val_loss: 8.6135 - val_mean_absolute_percentage_error: 118.7749
Epoch 2/50
35/35 [==============================] - 0s 8ms/step - loss: 0.2932 - mean_absolute_percentage_error: 253817.0469 - val_loss: 8.7382 - val_mean_absolute_percentage_error: 126.0864
Epoch 3/50
35/35 [==============================] - 0s 7ms/step - loss: 0.3294 - mean_absolute_percentage_error: 52.0452 - val_loss: 8.4217 - val_mean_absolute_percentage_error: 130.5384
Epoch 4/50
35/35 [==============================] - 0s 7ms/step - loss: 0.3166 - mean_absolute_percentage_error: 382315.0938 - val_loss: 8.4748 - val_mean_absolute_percentage_error: 111.3898
Epoch 5/50
35/35 [==============================] - 0s 8ms/step - loss: 0.2893 - mean_absolute_percentage_error: 250786.5625 - val_loss: 8.9979 - val_mean_absolute_percentage_error: 106.7632
Epoch 6/50
35/35 [==============================

In [ ]:
pka_model.fit(X_pka_train, y_pka_train, epochs=50, batch_size=32,validation_split=0.15,validation_data=None,verbose=1)
hfus_model.fit(X_hfus_train, y_hfus_train, epochs=50, batch_size=32,validation_split=0.15,validation_data=None,verbose=1)

Epoch 1/50
35/35 [==============================] - 0s 5ms/step - loss: 0.2720 - mean_absolute_percentage_error: 485595.0625 - val_loss: 9.9213 - val_mean_absolute_percentage_error: 130.6007
Epoch 2/50
35/35 [==============================] - 0s 4ms/step - loss: 0.2560 - mean_absolute_percentage_error: 430877.3438 - val_loss: 9.0969 - val_mean_absolute_percentage_error: 117.0430
Epoch 3/50
35/35 [==============================] - 0s 5ms/step - loss: 0.2626 - mean_absolute_percentage_error: 98431.0469 - val_loss: 8.0997 - val_mean_absolute_percentage_error: 94.8516
Epoch 4/50
35/35 [==============================] - 0s 4ms/step - loss: 0.2666 - mean_absolute_percentage_error: 271383.4688 - val_loss: 8.7725 - val_mean_absolute_percentage_error: 119.6940
Epoch 5/50
35/35 [==============================] - 0s 4ms/step - loss: 0.2019 - mean_absolute_percentage_error: 204420.4688 - val_loss: 7.8763 - val_mean_absolute_percentage_error: 114.2864
Epoch 6/50
35/35 [=============================

In [ ]:
y_pred_nn_pka_valid = pka_model.predict(X_pka_valid)
y_pred_nn_poly_hfus_valid = hfus_model.predict(X_hfus_valid)
print('Coefficient of determination for validation case: {}'.format(r2_score(y_pka_valid, y_pred_nn_pka_valid)))
print('Coefficient of determination for validation case: {}'.format(r2_score(y_hfus_valid, y_pred_nn_poly_hfus_valid)))

Coefficient of determination for validation case: 0.37723711002278915
Coefficient of determination for validation case: 0.564997323645672


In [ ]:
y_pred_nn_pka_test = pka_model.predict(X_pka_test)
y_pred_nn_hfus_test = hfus_model.predict(X_hfus_test)
print('Coefficient of determination for test case: {}'.format(r2_score(y_pka_test, y_pred_nn_pka_test)))
print('Coefficient of determination for test case: {}'.format(r2_score(y_hfus_test,y_pred_nn_hfus_test)))

Coefficient of determination for test case: 0.42638738061488224
Coefficient of determination for test case: 0.7488256719969884


In [ ]:
#np.max(score_hfus)

In [ ]:
#n[np.argmax(score_hfus)]